In [1]:


import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))



2023-09-02 14:08:09.270610: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 14:08:09.303805: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 14:08:09.305153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 14:08:09.925267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)
    
    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰⚽️✌�￰৷￰'''
    
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [3]:
train_url = 'https://raw.githubusercontent.com/SyedT1/NLP/main/FromScratch/error%20detection/data/data/train_data.csv'
test_url = 'https://raw.githubusercontent.com/SyedT1/NLP/main/FromScratch/error%20detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('stopwords_bangla.xlsx',index_col=False)
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [4]:
df_train['Comment'] = df_train.Comment.apply(lambda x: preprocessing(str(x)))
df_test['Comment'] = df_test.Comment.apply(lambda x:preprocessing(str(x)))
df = pd.concat([df_train,df_test],ignore_index = True)

In [5]:
set(df['Category'].values)

{'Code Switching', 'Grammatical', 'Multiple Errors', 'Spelling', nan}

In [6]:
def encode(s):
    d = {
        "Code Switching":0,
        "Grammatical":1,
        "Multiple Errors":2,
        "Spelling":3
    }
    if s in d:
        return d[s]
    else:
        return 4
df['Category'] = df.Category.apply(lambda x: encode(x))
df_train['Category'] = df_train.Category.apply(lambda x: encode(x))
df_test['Category'] = df_test.Category.apply(lambda x: encode(x))

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
data = df_train['Comment']
labels = df_train['Category']
# Tokenize text data
max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['Comment'])
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create CNN-LSTM model
model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=16, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
402/402 [==============================] - 10s 21ms/step - loss: 1.1052 - accuracy: 0.5929 - val_loss: 0.9815 - val_accuracy: 0.6362
Epoch 2/10
402/402 [==============================] - 8s 21ms/step - loss: 0.9378 - accuracy: 0.6571 - val_loss: 0.9403 - val_accuracy: 0.6486
Epoch 3/10
402/402 [==============================] - 9s 21ms/step - loss: 0.8055 - accuracy: 0.6943 - val_loss: 0.9762 - val_accuracy: 0.6269
Epoch 4/10
402/402 [==============================] - 9s 21ms/step - loss: 0.6669 - accuracy: 0.7424 - val_loss: 1.1123 - val_accuracy: 0.6045
Epoch 5/10
402/402 [==============================] - 8s 21ms/step - loss: 0.5375 - accuracy: 0.7914 - val_loss: 1.2753 - val_accuracy: 0.5976
Epoch 6/10
402/402 [==============================] - 8s 21ms/step - loss: 0.4141 - accuracy: 0.8461 - val_loss: 1.5179 - val_accuracy: 0.5553
Epoch 7/10
402/402 [==============================] - 9s 21ms/step - loss: 0.3218 - accuracy: 0.8808 - val_loss: 1.7341 - val_accuracy: 0.562

In [13]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))

63/63 [==============================] - 1s 8ms/step
Test set
  Loss: 2.539
  Accuracy: 0.554
  Macro Recall: 0.400
  Macro Precision: 0.412
